## Arnoldi's methods

In [380]:
import LinearAlgebra
const la = LinearAlgebra
MAX_ITER = 100;

In [381]:
A = [4. -1 0 -1 0 0;
     -1 4 -1 0 -1 0;
     0 -1 4. 0 0 -1;
     -1 0 0. 4 -1 0;
     0 -1 0 -1 4 -1;
     0 0 -1 0. -1 4]

b = [0, 5, 0, 6, -2, 6.]

x0 = zeros(size(b));
# n
v = [1., 0, 2, 0, 1, 1];

### Arnoldi
Input: $A\in\mathbb{R}^{n\times m}=[a_1,...,a_m], v\in\mathbb{R}^{n}$ any vector

In [382]:
function arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    n = size(A, 1)
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        for i = 1:j
            H[i, j] = la.dot(A*V[:, j], V[:, i])
        end
        w = A*V[:, j] - V[:, 1:j]*H[1:j, j]
        H[j+1, j] = la.norm(w)
        if H[j+1, j] == 0.0
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    # delete last row of H and last column of V
    return H[1:m, :], V[:, 1:m]
end

arnoldi (generic function with 1 method)

In [383]:
m = 3
H, V = arnoldi(A, v, m);
display(H)
display(V)

3×3 Matrix{Float64}:
 3.14286  1.92195  -2.77556e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556

6×3 Matrix{Float64}:
 0.377964   0.168563    0.289595
 0.0       -0.786629    0.191845
 0.755929   0.140469   -0.282287
 0.0       -0.393314    0.0959226
 0.377964  -0.0280939   0.750937
 0.377964  -0.421408   -0.475959

### Arnoldi-Modified Gram-Schmidt

In [384]:
function arnoldi_modified(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    
    H = zeros((m + 1, m))
    V = zeros((n, m + 1))
    
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = 1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    # delete last row of H and last column of V
    return H[1:m, :], V[:, 1:m]
end

arnoldi_modified (generic function with 1 method)

In [385]:
m = 3
H, V = arnoldi_modified(A, v, m)
display(H)
display(V)

3×3 Matrix{Float64}:
 3.14286  1.92195  -2.9976e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556

6×3 Matrix{Float64}:
 0.377964   0.168563    0.289595
 0.0       -0.786629    0.191845
 0.755929   0.140469   -0.282287
 0.0       -0.393314    0.0959226
 0.377964  -0.0280939   0.750937
 0.377964  -0.421408   -0.475959

### Householder Arnoldi

In [437]:
function householder_arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m < n
    # Output: nx(m+1) matrix H and an nxm orthonormal matrix V
    n = size(A, 1)
    I = 1.0 * Matrix(la.I, n, n) 
    V = zeros((n, m + 1))
    H = zeros((n, m + 1))
    Z = zeros((n, m + 1))

    Z[:, 1] = v
    Q = I
    for j = 1:m + 1
        # w calculus
        val_sign = Z[j, j] >= 0.0 ? 1.0 : -1.0
        beta = val_sign * la.norm(Z[j:end, j])
        z = zeros(n)
        z[j] = beta - Z[j, j] + eps()
        for i = j + 1:n
            z[i] = -Z[i, j]
        end
        w = z / la.norm(z)
       
        P = I - 2 * w * w'
        # h_{j-1}
        H[:, j] = P * Z[:, j] 
        Q = P * Q
        V[:, j] = Q'[:, j]
        if j <= m
            Z[:, j + 1] = Q * A * V[:, j]
        end
    end
    # delete the column 0 and return a (m+1)xm matrix H
    return H[1:m + 1, 2:m + 1], V
end

householder_arnoldi (generic function with 1 method)

In [448]:
m = 5
H, V = householder_arnoldi(A, v, m)
show(IOContext(stdout, :limit => false), "text/plain", H)
display(V)

6×5 Matrix{Float64}:
  3.14286      -1.92195       0.0           2.22045e-16   4.44089e-16
 -1.92195       4.6472        1.18932      -2.77556e-16  -2.22045e-16
 -2.77556e-16   1.18932       3.95556      -0.812257      1.11022e-16
  2.77556e-16  -1.07499e-16  -0.812257      3.67305       0.114091
 -5.55112e-17  -6.55942e-18   3.09757e-16   0.114091      3.58133
  2.22045e-16   3.46945e-18  -6.93889e-18   6.73073e-16   5.55112e-16

6×6 Matrix{Float64}:
 0.377964  -0.168563   -0.289595   -0.585252    0.390312   0.5
 0.0        0.786629   -0.191845    0.228787    0.540433  -9.4369e-16
 0.755929  -0.140469    0.282287    0.128662    0.2502    -0.5
 0.0        0.393314   -0.0959226  -0.699892   -0.310248  -0.5
 0.377964   0.0280939  -0.750937    0.313906   -0.440352  -1.66533e-16
 0.377964   0.421408    0.475959    0.0140224  -0.45036    0.5

It is verified that $AV_m = V_{m+1}H$

In [449]:
isapprox(A*V[:, 1:m], V*H)

true

## Arnoldi’s Method for Linear Systems (FOM)

In [440]:
function FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64)
    n = size(A, 1)
    r = b - A*x0
    beta = la.norm(r)
    v = r/beta
    H, V = arnoldi_modified(A, v, m)
    return H, V, beta
end

FOM (generic function with 1 method)

In [441]:
m = 3
H, V, beta = FOM(A, b, x0, m)
y = la.inv(H)[:, 1] * beta
x = x0 + V*y

6-element Vector{Float64}:
 0.9907832820475926
 1.9916351376539796
 0.9907832820475926
 2.0053245062998073
 1.01182460603722
 2.0053245062998073

In [450]:
function restarted_FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-6)
    println("x[0] = $x0")
    for i=1:MAX_ITER
        r = b - A*x0
        beta = la.norm(r)
        v = r/beta

        H, V = arnoldi(A, v, m)
        H = H[1:m, :]
        V = V[:, 1:m]
        y = la.inv(H)[:, 1] * beta
        x = x0 + V*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

restarted_FOM (generic function with 2 methods)

In [452]:
m = 2
restarted_FOM(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [1.0223756706686238, 1.6864518923080627, 1.0223756706686238, 2.0609195678848975, 0.8310997762432939, 2.0609195678848975]
x[2] = [0.984849223774084, 1.9693401764671083, 0.984849223774084, 1.9683804438298635, 0.9830857423107029, 1.9683804438298635]
x[3] = [0.9999872854527297, 1.9948107917617135, 0.9999872854527297, 2.0011111663744794, 0.9977971748604653, 2.0011111663744794]
x[4] = [0.999767337695896, 1.9995208450252253, 0.999767337695896, 1.999490305397517, 0.9996996548524394, 1.999490305397517]
x[5] = [0.9999917048166211, 1.9999124703172824, 0.9999917048166213, 2.0000198650717804, 0.9999742459912793, 2.000019865071781]
x[6] = [0.9999963180209491, 1.9999922216157726, 0.9999963180209492, 1.9999915531898904, 0.999994404480674, 1.9999915531898906]
x[7] = [0.9999997021856807, 1.9999985010572652, 0.9999997021856807, 2.0000003403577056, 0.9999997579690995, 2.0000003403577056]
x[8] = [0.9999999392404859, 1.9999998669860726, 0.9999999392404859, 1.9999

### Incomplete Orthogonalization Process

In [393]:
function IOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64)
    n = size(A, 1)
    V = zeros((n, m+1))
    H = zeros((m+1, m))
    
    r = b - A*x0
    beta = la.norm(r)
    V[:, 1] = r/beta
    for j = 1:m
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
    end
    return H[1:m, :], V[:, 1:m], beta
end

IOM (generic function with 1 method)

In [394]:
m = 5
k = 3
H, V = IOM(A, b, x0, m, k)
display(H)
display(V)

5×5 Matrix{Float64}:
 4.67327  2.21877   3.747e-15   0.0          0.0
 2.21877  3.35573   0.690815   -4.07868e-14  0.0
 0.0      0.690815  4.32887     0.217999     2.94568
 0.0      0.0       0.217999    3.64213      0.0487366
 0.0      0.0       0.0         2.80489e-14  1.76082

6×5 Matrix{Float64}:
  0.0       -0.493309   -0.238968  -0.446699  -0.308737
  0.497519  -0.0612751   0.7893    -0.354578   0.134578
  0.0       -0.493309   -0.238968  -0.446699  -0.308737
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391
 -0.199007  -0.702       0.337983   0.59444    0.0949961
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391

### DIOM

In [460]:
function DIOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int, k:: Int, tol=1.0e-6)
    n = size(A, 1)
    V = zeros(n, m + 1)
    H = zeros(m + 1, m)
    P = zeros(n, m)
    
    r0 = b - A*x0
    beta = la.norm(r0)
    zeta = beta
    V[:, 1] = r0/beta

    println("x[0] = $x0")
    for j=1:MAX_ITER
        w = A*V[:, j]
        # IOM block
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
        
        # update LU factorization
        L, U = la.lu(H[1:j, 1:j])
        if U[j ,j] < eps()
            break
        end
        zeta = j == 1 ? beta : -L[j, j-1]*zeta  
        
        i = j - k + 1
        vector_sum = i <= 0 ? zeros(m) : P[:, i:j-1]*U[i:j-1, j]  
        P[:, j] = 1/U[j, j]*(V[:, j] - vector_sum)
            
        x = x0 + zeta*P[:, j]
        
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

DIOM (generic function with 4 methods)

In [459]:
m = 6
k = 2
DIOM(A, b, x0, m, k)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.0223756706686236, 1.6864518923080625, 1.0223756706686236, 2.0609195678848975, 0.8310997762432935, 2.0609195678848975]
x[3] = [0.9907832820475928, 1.99163513765398, 0.9907832820475928, 2.0053245062998073, 1.0118246060372202, 2.0053245062998073]
x[4] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]
x[5] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]


## The Generalized Minimum Residual Method (GMRES)

In [397]:
function givens_transformation(H:: Matrix, i:: Int64)
    # H is a nxm Hessemberg matrix
    # i row
    n, m = size(H)
    upper_omega = 1.0 * Matrix(la.I, n, n) 
    hyp = sqrt(H[i, i]^2 + H[i + 1, i]^2)
    si = H[i + 1, i]/hyp
    ci = H[i, i]/hyp
    
    upper_omega[i, i] = ci
    upper_omega[i, i + 1] = si
    upper_omega[i + 1, i] = -si
    upper_omega[i + 1, i + 1] = ci
    return upper_omega 
end


givens_transformation (generic function with 1 method)

In [398]:
H = [1 4 2 3 5;
     3 4 1 7 1;
     0 2 3 4 0;
     0 0 1 3 1;
     0 0 0 3 5;
     0 0 0 0 4.]

n, m = size(H)
for i=1:m
    upper_omega = givens_transformation(H, i)
    H = upper_omega * H
end
H

6×5 Matrix{Float64}:
 3.16228  5.05964  1.58114  7.58947   2.52982
 0.0      3.2249   3.10087  2.97683   3.47297
 0.0      0.0      1.69842  3.98562  -1.63048
 0.0      0.0      0.0      3.10698   5.45867
 0.0      0.0      0.0      0.0       4.13314
 0.0      0.0      0.0      0.0       0.0

In [399]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, M = size(A)
    k = 1

    V = zeros((M, M+1))
    H = zeros((M+1, M))
    P = zeros((M, M))
    gamma = zeros(M)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for m=1:MAX_ITER
        w = A*V[:, m]
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V[:, m+1] = w/H[m+1, m]
        
        
        H[1:m, 1:m] = Qm*H[1:m, 1:m]

        s_m = s(m, H)
        c_m = c(m, H)

        gamma[m + 1] = -s_m*gamma[m]
        gamma[m] = c_m*gamma[m]
        H[m, m] = c_m*H[m, m] + s_m*H[m+1, m]
        i = m - k + 1
        sum = i > 0 ? P[:, i:m-1]*H[i:m-1, m] : zeros(M)
        
        P[:, m] = (V[:, m] - sum)/H[m, m]
        x = x0 + gamma[m]*P[:, m]
        
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 2 methods)

In [400]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, m = size(A)
    k = 1

    V = zeros((n, m+1))
    H = zeros((m+1, m))
    P = zeros((n, n))
    gamma = zeros(m)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for j=1:MAX_ITER
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
        
        for i = m-k:m-1
            H = givens_transformation(H, i) * H
        end

        hyp = sqrt(H[j, j]^2 + H[j + 1, j]^2)
        sj = H[j + 1, j]/hyp
        cj = H[j, j]/hyp

        gamma[j + 1] = -sj*gamma[j]
        gamma[j] = cj*gamma[j]
        H[j, j] = cj*H[j, j] + sj*H[j+1, j]
        
        P[:, j] = (V[:, j] - P[:, m-k:m-1]*H[m-k:m-1, j])/H[j, j]
        x = x0 + gamma[j]*P[:, j]
        
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 2 methods)

In [401]:
x = DQGMRES(A, b, x0)
x

LoadError: BoundsError: attempt to access 6-element Vector{Float64} at index [7]